In [1]:
from gurobipy import *  # import the optimize solver Gurobi

In [2]:
number_of_drink = 2 # Set the index i for number types of drink(Thunder, Viper)
number_of_material = 3 # Set the index j for number types of material(Mixer, Vodka, Whiskey)
m = Model() # Import and create the model

Using license file C:\Users\chaoz\gurobi_licence\gurobi.lic
Academic license - for non-commercial use only


In [3]:
# Set the input Parameter: 
selling_price = [5.4, 4.2] # Selling price of drink type i
material_needed = [[0, 0.2, 0], [0, 0.1, 0.1]] # Minimum fraction of material needed fro each drink
material_lim = [1000, 200, 100]  # Availability of material j

In [4]:
# Set the Variable list: Ounces of material j to be allocated to drink type i
# Set the variable to integer number
nx = [[0, 0, 0], [0, 0, 0]]
for i in range(number_of_drink):
    for j in range(number_of_material):
        nx[i][j] = m.addVar(vtype=GRB.INTEGER, name='nx{}'.format(str(i + 1) + str(j + 1)))

# Caculate total ounces allocated to each typy of drink
t1 = []
for i in range(number_of_drink):
    t1.append(sum([nx[i][j] for j in range(number_of_material)]))
    
# Caculate total ounces allocated to each typy of drink
t2 = []
for j in range(number_of_material):
    t2.append(sum([nx[i][j] for i in range(number_of_drink)]))

In [5]:
m.update()
print(t2)

[<gurobi.LinExpr: nx11 + nx21>, <gurobi.LinExpr: nx12 + nx22>, <gurobi.LinExpr: nx13 + nx23>]


In [6]:
# Set the Maximize Obijective: Total Revenue
m.setObjective(quicksum([t1[i] * selling_price[i] for i in range(number_of_drink)]) ,  GRB.MAXIMIZE)

In [7]:
# Set Non Negative number of order
for i in range(number_of_drink):
     for j in range(number_of_material):
            c1 = m.addConstr(nx[i][j]  >= 0)

In [10]:
# Satisfy minimum percentage of each material units of in each drink type
c2 = []
for i in range(number_of_drink):
    for j in range(number_of_material):
        c2 = m.addConstr( (t1[i] * material_needed[i][j] ) <= nx[i][j])

In [11]:
# Satisfy maximum availability of materials
c3 = []
for j in range(number_of_material):
    c3 = m.addConstr(t2[j] <= material_lim[j])

In [12]:
# Run the optimize solver
m.optimize()

Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (win64)
Optimize a model with 18 rows, 6 columns and 30 nonzeros
Model fingerprint: 0x89903354
Variable types: 0 continuous, 6 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e-01, 1e+00]
  Objective range  [4e+00, 5e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 1e+03]
Found heuristic solution: objective 1620.0000000
Presolve removed 13 rows and 1 columns
Presolve time: 0.00s
Presolved: 5 rows, 5 columns, 13 nonzeros
Found heuristic solution: objective 5400.0000000
Variable types: 0 continuous, 5 integer (0 binary)

Root relaxation: objective 6.300000e+03, 3 iterations, 0.39 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    6300.0000000 6300.00000  0.00%     -    1s

Explored 0 nodes (3 simplex iterations) in 1.34 seconds
Thread count was 8 (of 8 available processo

In [13]:
# Get the Optimal Solution for X
m.printAttr('X')


    Variable            X 
-------------------------
        nx11          560 
        nx12          140 
        nx21          440 
        nx22           60 
        nx23          100 


In [14]:
# Get the Optimal Objective Value
m.ObjVal

6300.0